In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Policy

In [5]:
from src.faithnet import network
import tensorflow as tf
import gym

In [8]:
class replay_stacker():
    def __init__(self, columns, window_length=100):
        self._data = np.zeros((window_length, columns))
        self.capacity = window_length
        self.size = 0
        self.columns = columns

    def update(self, x):
        self._add(x)

    def _add(self, x):
        if self.size == self.capacity:
            self._data = np.roll(self._data, -1)
            self._data[self.size-1, :] = x
        else:
            self._data[self.size, :] = x
            self.size += 1

    def data(self):
        return self._data[0:self.size, :]


class replay_memory_agent():
    def __init__(self, states_len, replay_window):
        self.states_len = states_len
        self.replay_window = replay_window

        # Initialize replay memory
        self.replay_memory = {'states': replay_stacker(self.states_len, self.replay_window),
                              'states_new': replay_stacker(self.states_len,self.replay_window),
                              'rewards': replay_stacker(1,self.replay_window),
                              'actions': replay_stacker(1,self.replay_window),
                              'terminal': replay_stacker(1,self.replay_window)}

    def replay_memory_update(self,
                             states,
                             states_new,
                             rewards,
                             actions,
                             terminal):

        self.replay_memory['rewards'].update(rewards)
        self.replay_memory['states'].update(states)
        self.replay_memory['states_new'].update(states_new)
        self.replay_memory['actions'].update(actions)
        self.replay_memory['terminal'].update(terminal)


def randombatch(sample_size, replay_size):
    indx = np.linspace(0, replay_size-1, sample_size)
    indx = np.random.choice(indx, sample_size, replace=False)
    indx.tolist()
    indx = list(map(int, indx))
    return indx

In [3]:
nn_critic = network(4, 2, [10, 10], 2, tf.Session())
nn_actor = network(4, 2, [10, 10], 2, tf.Session())
target_nn_critic = network(4, 2, [10, 10], 2, tf.Session())
target_nn_actor = network(4, 2, [10, 10], 2, tf.Session())

In [9]:
target_nn_actor.set_weights(nn_actor.get_weights())
target_nn_critic.set_weights(nn_critic.get_weights())
env = gym.make("CartPole-v1")
replay = replay_memory_agent(4, 10000)

[2017-11-23 21:14:46,461] Making new env: CartPole-v1


In [ ]:
training_state = np.zeros((32, 4))
training_new_state = np.zeros((32, 4))
training_actions = np.zeros((32, 1))
training_rewards = np.zeros((32, 1))
training_terminal = np.zeros((32, 1))

for i in range(0, 10):
    # Reset the environment
    done = False
    state = env.reset()
    state = np.asarray(state)
    state = np.reshape(state, (1, 4))
    # Exploration
    mean = 0.0
    deviation = 1.0
    while done:
        exploration_noise = np.random.normal(mean, deviation)
        exploration_noise = exploration_noise.reshape(1, 2)
        action = nn_actor.predict_on_batch(state) + exploration_noise
        
        act = np.argmax(action)
        # Take action 
        new_state, reward, done, _ = env.step(act)
        
        # Store the transitions
        replay.replay_memory_update(state, new_state, reward, act, done)
        
        # Pick a random batch
        indx = randombatch(32, len(replay.replay_memory['states'].data()))
        temp_state = replay.replay_memory["states"].data()
        training_state = temp_state[indx]
        temp_state = replay.replay_memory["states_new"].data()
        training_new_state = temp_state[indx]
        temp_state = replay.replay_memory["actions"].data()
        training_actions = temp_state[indx]
        temp_state = replay.replay_memory["rewards"].data()
        training_rewards = temp_state[indx]
        temp_state = replay.replay_memory["terminal"].data()
        training_terminal = temp_state[indx]
        
        target = nn_critic.predict_on_batch(training_state)
        q_values_train_next = 
        for i in range(32):
            action_idx = int(training_actions[i])
            if temp_terminal[i]:
                target[i][action_idx] = temp_rewards[i]
            else:
                target[i][action_idx] = temp_rewards[i] + 0.99 * np.amax(q_values_train_next[i])
